In [ ]:
import actions 
project_id  = 17
active_scenes = actions.get_active_scenes(project_id)
# print(active_scenes)
scenes_words_count_list = []
for scene in active_scenes:
    scene_text = scene.get("scene_text")
    no_words_in_scene = len(scene_text.split())
    scenes_words_count_list.append(no_words_in_scene)

print(scenes_words_count_list)
total_words_in_project = sum(scenes_words_count_list)
print(total_words_in_project)

In [ ]:

import funcs_supabase
import media
import requests
import random
import transcriber

def get_word_list_for_scene(project_id):
    """Fetches speech audio URL from the database and gets its subtitles."""

    speech_audio_data = funcs_supabase.select_data("video_creator_projects", "id", project_id)
    speech_audio_url = speech_audio_data[0]['speech_audio']
    print(f"Speech audio URL: {speech_audio_url}")


    # # Download the audio file into memory
    response = requests.get(speech_audio_url)
    rand_no = random.randint(0, 1000000000000000)
    filename = f"uploads/{rand_no}.mp3"
    # save to disk 
    with open(filename, "wb") as f:
        f.write(response.content)


    with open(filename, "rb") as audio_file_binary:
        print("about to get word segments")
        word_segments = transcriber.get_word_segments(audio_file_binary)
        word_list = transcriber.process_word_segments(word_segments)
        word_list = transcriber.fix_missing_time_stamps(word_list)

    return word_list
    #     blocks_of_words = generate_subtitle_blocks(word_list, characters)
    #     blocks_of_words = process_blocks(blocks_of_words)

project_id = 17
words_list = get_word_list_for_scene(project_id)
for word in words_list:
    print(word)

In [ ]:
from moviepy.editor import AudioFileClip

print(len(words_list))
print(total_words_in_project)
print(words_list)




last_word_end_time = words_list[-1]['end']
first_word_time = words_list[0]['start']
print(f"first word start time {first_word_time}")
print(f"last word end time {last_word_end_time}")

speech_audio = "composed_videos/project17-2/speech.mp3"
audio = AudioFileClip(speech_audio)
audio_duration = audio.duration # this will print the duration of the audio in seconds
print(f"audio duration {audio_duration}")
audio.close()







In [ ]:
# from moviepy.editor import AudioFileClip, ColorClip, concatenate_videoclips

# from moviepy.editor import CompositeVideoClip

# # Load your audio file
# speech_audio = "composed_videos/project17-2/speech.mp3"
# audio = AudioFileClip(speech_audio)


# # Create a red background video clip with the same duration as your audio
# red_background = ColorClip(size=(1920, 1080), color=(255, 0, 0)).set_duration(audio.duration)

# # Set the audio of the red background video clip to be your audio file
# red_video_with_audio = red_background.set_audio(audio)

# # Write the result to a file
# red_video_with_audio.write_videofile('red3.mp4', fps=24)

# # Close the clips to free up resources
# audio.close()
# red_background.close()
# red_video_with_audio.close()




In [ ]:
import actions
import funcs_supabase

def process_project_scenes(project_id):

    scene_timings_raw = []

    active_scenes = actions.get_active_scenes(project_id)

    # Check if active_scenes is None or empty before proceeding
    if not active_scenes:
        return "No active scenes available for this project."

    total_words_in_project = 0

    for scene_counter, scene in enumerate(active_scenes):
        scene_text = scene.get("scene_text", "")
        scene_id = scene.get("id")

        # If scene_text or scene_id is missing, skip to next iteration
        if not scene_text or scene_id is None:
            continue

        # print(scene_text)
        words_in_scene = scene_text.split()
        no_words_in_scene = len(words_in_scene)

        # print(f"scene has {no_words_in_scene} words")
        # print(f"the first word is {words_in_scene[0]}")
        # print(f"the last word is {words_in_scene[-1]}")

        total_words_in_project += no_words_in_scene
        first_word_location = total_words_in_project - no_words_in_scene
        last_word_location = total_words_in_project - 1

        # print(f"first word location: {first_word_location}")
        # print(f"last word location: {last_word_location}")

        try:
            first_word_from_words_list = words_list[first_word_location]
            last_word_from_words_list = words_list[last_word_location]
        except IndexError:
            # Handle this error if word locations are out of bounds
            continue

        # print(f"first word from words list: {first_word_from_words_list}")
        # print(f"last word from words list: {last_word_from_words_list}")

        if scene_counter == 0:
            words_list[first_word_location]["start"] = 0

        scene_length = last_word_from_words_list["end"] - first_word_from_words_list["end"]
        scene_length = round(scene_length, 3)

        data_to_update = {
            "length": scene_length,
            "start": first_word_from_words_list["start"],
            "end": last_word_from_words_list["end"]
        }

        # funcs_supabase.update_data("scenes", scene_id, {"timings": data_to_update})
        # print(f"scene length: {scene_length}")
        # print("\n")

        scene_timings_raw.append(data_to_update)

    return scene_timings_raw
    
    

# Usage
project_id = 17
scene_timings_raw = process_project_scenes(project_id)


In [ ]:
def process_scene_timings(scene_timings_raw):
    
    scene_timings_processed = []
    for i, scene in enumerate(scene_timings_raw):

        # print(scene)

        # first scene starts at 0
        if i == 0:
            scene["start"] = 0
       
        # if not the first scene, the start time is the end time of the previous scene
        else:
            scene["start"] = scene_timings_raw[i-1]["end"]
        
        # if not the last scene, the end time is the start time of the next scene
        if i < len(scene_timings_raw) - 1:
            scene["end"] = scene_timings_raw[i+1]["start"]


        # if the last scene, end at the end of the audio
        else:
            scene["end"] = audio_duration

        
        
        scene["start"] = round(scene["start"], 3)
        scene["end"] = round(scene["end"], 3)

        # recalculate the length 
        scene["length"] = scene["end"] - scene["start"]

        # round the length to 3 decimal places
        scene["length"] = round(scene["length"], 3)


        scene_timings_processed.append(scene)

    return scene_timings_processed


scene_timings_processed = process_scene_timings(scene_timings_raw)
for scene in scene_timings_processed:
    print(scene)

In [ ]:
def update_timings_database(processed_scene_timings, project_id):
    active_scenes = actions.get_active_scenes(project_id)

    if len(active_scenes) != len(processed_scene_timings):
        print("The number of scenes in the database and the number of scenes in the processed scene timings do not match.")
        return
    
    for i, scene in enumerate(processed_scene_timings):

        data_to_update = {
        "length": scene["length"],
        "start": scene["start"],
        "end": scene["end"]
        }

        scene_id = active_scenes[i]["id"]
        print(f"updating scene {scene_id} with {data_to_update}")
        update = funcs_supabase.update_data("scenes", scene_id, {"timings": data_to_update})
        print(update)
        print("\n")

# Usage
project_id  = 17
update_timings_database(scene_timings_processed, project_id)

In [ ]:
import actions
import funcs_supabase
project_id  = 19
active_scenes = actions.get_active_scenes(project_id)


total_words_in_project = 0
scene_counter = 0
for scene in active_scenes:
    scene_text = scene.get("scene_text")
    scene_id = scene.get("id")
    print(scene_text)
    words_in_scene = scene_text.split()
    no_words_in_scene = len(words_in_scene)
    print(f"scene has {no_words_in_scene} words")
    print(f"the first word is {words_in_scene[0]}")
    print(f"the last word is {words_in_scene[-1]}")

    total_words_in_project += no_words_in_scene
    first_word_location = total_words_in_project - no_words_in_scene
    last_word_location = total_words_in_project - 1

    print(f"first word location: {first_word_location}")
    print(f"last word location: {last_word_location}")

    first_word_from_words_list = words_list[first_word_location]
    last_word_from_words_list = words_list[last_word_location]

    print(f"first word from words list: {first_word_from_words_list}")
    print(f"last word from words list: {last_word_from_words_list}")



    if scene_counter == 0:
        words_list[first_word_location]["start"] = 0

    scene_length = last_word_from_words_list["end"] - first_word_from_words_list["end"]
    scene_length = round(scene_length, 3)

    data_to_update = {
        "length": scene_length,
        "start" : first_word_from_words_list["start"],
        "end" : last_word_from_words_list["end"]
    }

    funcs_supabase.update_data("scenes", scene_id, {"timings": data_to_update})
    print(f"scene length: {scene_length}")

    print("\n")

    scene_counter += 1


# print(scenes_words_count_list)
# total_words_in_project = sum(scenes_words_count_list)
# print(total_words_in_project)
